> As the number of attributes becomes large, exploratory analysis of the all the predictors may be infeasible and concentrating on those with strong relationships with the outcome may be an effective triaging strategy. 

In [2]:
library(AppliedPredictiveModeling)
data(solubility)

In [2]:
ls()

[1] "solTestX"       "solTestXtrans"  "solTestY"       "solTrainX"     
[5] "solTrainXtrans" "solTrainY"

In [4]:
str(solTrainXtrans)
str(solTrainY)

'data.frame':	951 obs. of  228 variables:
 $ FP001            : int  0 0 1 0 0 1 0 1 1 1 ...
 $ FP002            : int  1 1 1 0 0 0 1 0 0 1 ...
 $ FP003            : int  0 0 1 1 1 1 0 1 1 1 ...
 $ FP004            : int  0 1 1 0 1 1 1 1 1 1 ...
 $ FP005            : int  1 1 1 0 1 0 1 0 0 1 ...
 $ FP006            : int  0 1 0 0 1 0 0 0 1 1 ...
 $ FP007            : int  0 1 0 1 0 0 0 1 1 1 ...
 $ FP008            : int  1 1 1 0 0 0 1 0 0 0 ...
 $ FP009            : int  0 0 0 0 1 1 1 0 1 0 ...
 $ FP010            : int  0 0 1 0 0 0 0 0 0 0 ...
 $ FP011            : int  0 1 0 0 0 0 0 0 1 0 ...
 $ FP012            : int  0 0 0 0 0 1 0 1 0 0 ...
 $ FP013            : int  0 0 0 0 1 0 1 0 0 0 ...
 $ FP014            : int  0 0 0 0 0 0 1 0 0 0 ...
 $ FP015            : int  1 1 1 1 1 1 1 1 1 1 ...
 $ FP016            : int  0 1 0 0 1 1 0 1 0 0 ...
 $ FP017            : int  0 0 1 1 0 0 0 0 1 1 ...
 $ FP018            : int  0 1 0 0 0 0 0 0 0 0 ...
 $ FP019            : int  1 0 0 0 1 0 1

NULL

 num [1:951] -3.97 -3.98 -3.99 -4 -4.06 -4.08 -4.08 -4.1 -4.1 -4.11 ...


In [5]:
cor(solTrainXtrans$NumCarbon,solTrainY)

[1] -0.6067917

`Numeric Outcomes`

[A worked example: evaluating the relationship between two hormones](http://content.csbs.utah.edu/~rogers/datanal/labprj/bootstrap/index.html)

In [6]:
#Spearman Rank Correlation
cor(solTrainX$NumCarbon,solTrainY)
cor.test(solTrainX$NumCarbon,solTrainY, method="spearman")

[1] -0.5827611

Warning message in cor.test.default(solTrainX$NumCarbon, solTrainY, method = "spearman"):
"Cannot compute exact p-value with ties"


	Spearman's rank correlation rho

data:  solTrainX$NumCarbon and solTrainY
S = 239900000, p-value < 2.2e-16
alternative hypothesis: true rho is not equal to 0
sample estimates:
       rho 
-0.6735911 
